<div style="border: 2px solid #e74c3c; padding: 10px; border-radius: 5px; text-align: center; background-color: #fdecea; color: #c0392b; font-weight: bold;">
  Prompt Engineering
</div>

Le but de ce notebook est de tester différents principe du prompt engineering et de comparer les résultats.

> **Le prompt Engineering :** est le processus de conception des 𝐏𝐫𝐨𝐦𝐩𝐭s efficaces qui suscitent des résultats pertinents et de haute qualité de la part des llm.
>
> **Le prompt :** est une entrée de texte qui guide le comportement d'un LLM pour générer une réponse. Ex : Rédige un poème sous le thème mathématiques.

Quelques principes de base rendront votre prompt efficace : 
1. 𝐋𝐞𝐬 𝐢𝐧𝐬𝐭𝐫𝐮𝐜𝐭𝐢𝐨𝐧𝐬 𝐝𝐚𝐧𝐬 𝐯𝐨𝐭𝐫𝐞 𝐩𝐫𝐨𝐦𝐩𝐭 𝐝𝐨𝐢𝐯𝐞𝐧𝐭 ê𝐭𝐫𝐞 𝐜𝐥𝐚𝐢𝐫𝐞𝐬 : 

le but de la tâche, le format de retour, les contraintes et les limitations, le contexte de la tâche.

2. 𝐒é𝐩𝐚𝐫𝐞𝐫 𝐝𝐞𝐬 𝐭â𝐜𝐡𝐞𝐬 𝐜𝐨𝐦𝐩𝐥𝐞𝐱𝐞𝐬 𝐞𝐧 𝐝𝐞𝐬 𝐦𝐢𝐧𝐢 𝐩𝐫𝐨𝐦𝐩𝐭𝐬

3. 𝐃𝐞𝐦𝐚𝐧𝐝𝐞𝐫 𝐮𝐧𝐞 𝐣𝐮𝐬𝐭𝐢𝐟𝐢𝐜𝐚𝐭𝐢𝐨𝐧 au llm : 

Cela permet de vérifier si le llm hallucine ou pas.

4. 𝐑é𝐩é𝐭𝐞𝐫 𝐥𝐞𝐬 𝐢𝐧𝐬𝐭𝐫𝐮𝐜𝐭𝐢𝐨𝐧𝐬 à 𝐥𝐚 𝐟𝐢𝐧 𝐝𝐮 𝐩𝐫𝐨𝐦𝐩𝐭 : 

Les llms souffrent du biais de récence. Ils donnent plus de poids aux informations qui apparaissent à la fin du prompt et ont tendance à oublier les informations du début. Répéter l'instruction à la fin diminue fortement ce biais.

5. 𝐔𝐭𝐢𝐥𝐢𝐬𝐞𝐫 𝐝𝐞𝐬 𝐝é𝐥𝐢𝐦𝐢𝐭𝐞𝐮𝐫𝐬 : Cela améliore le formatage de votre prompt et vous permet de bien délimiter les parties de votre prompt.

En plus de ces principes de base, il existe des méthodes plus évoulées du prompt Engineering :

1. 𝐅𝐞𝐰-𝐒𝐡𝐨𝐭 𝐏𝐫𝐨𝐦𝐩𝐭𝐢𝐧𝐠 : il s'agit d'une technique de prompt engineering qui consiste à montrer au LLM quelques exemples (ou shots) des résultats désirés.

2. 𝐂𝐡𝐚𝐢𝐧 𝐨𝐟 𝐭𝐡𝐨𝐮𝐠𝐡𝐭 (𝐂𝐨𝐓) : il s'agit d'une technique qui permet des capacités de raisonnement complexes grâce à des étapes de raisonnement intermédiaires.

3. 𝐑𝐞𝐀𝐜𝐭 (𝐑𝐞𝐚𝐬𝐨𝐧 𝐚𝐧𝐝 𝐀𝐜𝐭 ) : il permet aux LLMs d'interagir avec des outils externes pour récupérer des informations supplémentaires qui conduisent à des réponses plus fiables et plus factuelles.


In [1]:
import os
from dotenv import load_dotenv
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain.schema import HumanMessage, SystemMessage
from langchain.agents import AgentExecutor, load_tools, initialize_agent
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.tools.render import render_text_description

load_dotenv()

True

In [2]:
# Hugging face api token
openai_api_key = os.environ['HUGGINGFACEHUB_API_TOKEN']

### Initialisation du modèle huggingface

In [15]:
llm = HuggingFaceEndpoint(
    repo_id= "mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    max_new_tokens=1024,
    do_sample=True,
    #repetition_penalty=1.1,
    temperature=0.95,
    top_p=0.95
    
)

In [16]:
chat_model = ChatHuggingFace(llm=llm)

In [17]:
def invoke_llm(system_message: str, user_message: str):
    """Invoke LLM

    Args:
        system_message (str): The system message
        user_message (str): The user message
    """
    
    message = [
        SystemMessage(content=system_message),
        HumanMessage(content=user_message)
    ]

    # Invoking the chat model
    response=chat_model.invoke(message)

    return response
    

### Les instructions doivent être claires.

In [18]:
system_message = """
You are an AI assistant that helps human by propose food recipe.
You will be provided with a text and the name of the meal. Then provide the recipe.

## Instructions
1. Give the results in this format :
     1. ingredients
        - Ingredient 1
        - Ingredient 2
     2. preparation
        - Chop Onions
        - Chop tomatoes
     3. Cooking
        - turn on the oven
        - bake the meal
Text: 
"""

In [19]:
instructions = """
Please provide me the recipe of Eru meal ?

"""

In [20]:
response = invoke_llm(system_message=system_message, user_message=instructions)
print(response.content)

1. Ingredients:
   - 2 cups of Eru grain
   - 2 cups of water
   - 2 tablespoons of oil
   - 1 large onion, finely chopped
   - 3-4 cloves of garlic, minced
   - 2 cups chopped Eru leaves
   - Salt to taste
   - Pepper to taste

2. Preparation:
   - Rinse the E


Le eru et watafufu est un plat traditionnel camerounais. Le LLM semble pas avoir compris car la tâche est complexe. On va diviser la tâche en sous tâches pour qu'il comprennne mieux.

In [21]:
system_message_justification = """
You are an AI assistant that helps human by propose food recipe.
You will be provided with a text and the name of the meal. Then provide the recipe.

## Instructions
1. You will first try to detect the country of origin of the meal.
2. Give the results in this format :
     1. ingredients
        - Ingredient 1
        - Ingredient 2
     2. preparation
        - Chop Onions
        - Chop tomatoes
     3. Cooking
        - turn on the oven
        - bake the meal

Text: 
"""

In [22]:
response = invoke_llm(system_message=system_message_justification, user_message=instructions)
print(response.content)

Country of Origin: Not specified (The name Eru does not correspond to a known traditional meal from a specific country. I will provide a general recipe for a meal similar to a casserole.)

Recipe:

ingredients
- 1 pound ground beef


Il semble toujours pas connaitre le eru, nous allons lui donner des outils externes vie le framework ReAct.

## ReAct

In [23]:
tools = load_tools(["serpapi"], llm=llm)

In [24]:
from langchain.agents import AgentType, initialize_agent

In [29]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
    handle_parsing_errors=True
)

In [30]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
instructions = """
What is the recipee of Eru meal ?
"""

In [35]:
agent.invoke(instructions)



> Entering new AgentExecutor chain...

Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:

KeyboardInterrupt: 